In [ ]:
!pip install gym[atari]
!pip install gym[accept-rom-license]

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import gym
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve, gaussian

import os
import io
import base64
import time
import glob
from IPython.display import HTML
import torch.nn.functional as F
from gym.wrappers import AtariPreprocessing
from gym.wrappers import FrameStack
from gym.wrappers import TransformReward


In [ ]:
def make_env(env_name, clip_rewards = True, seed = None):
	# complete this function which returns an object 'env' using gym module
	# Use AtariPreprocessing, FrameStack, TransformReward(based on the clip_rewards variable passed in the arguments of the function), check their usage from internet
	# Use FrameStack to stack 4 frames
	# TODO
  env = gym.make(env_name)
  env = AtariPreprocessing(env)
  env = FrameStack(env, num_stack=4)
  if clip_rewards:
        env = TransformReward(env, lambda r: np.sign(r))
  if seed is not None:
        env.seed(seed)
  return env

# Initialize the device based on CUDA availability
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


In [ ]:
# Next we create a class DQNAgent which is the class containing the neural network, This class is derived from nn.Module

class DQNAgent(nn.Module):
    def __init__(self, state_shape, n_actions, epsilon=0):

        super().__init__()
        self.epsilon = epsilon
        self.n_actions = n_actions
        self.state_shape = state_shape

        state_dim = state_shape[0]
        # a simple NN with state_dim as input vector (inout is state s)
        # and self.n_actions as output vector of logits of q(s, a)
        self.network = nn.Sequential()
        self.network.add_module('conv1', nn.Conv2d(4,16,kernel_size=8, stride=4))
        self.network.add_module('relu1', nn.ReLU())
        self.network.add_module('conv2', nn.Conv2d(16,32,kernel_size=4, stride=2))
        self.network.add_module('relu2', nn.ReLU())
        self.network.add_module('flatten', nn.Flatten())
        self.network.add_module('linear3', nn.Linear(2592, 256)) #2592 calculated above
        self.network.add_module('relu3', nn.ReLU())
        self.network.add_module('linear4', nn.Linear(256, n_actions))

        self.parameters = self.network.parameters

    def forward(self, state_t):
        # pass the state at time t through the newrok to get Q(s,a)
        qvalues = self.network(state_t)
        return qvalues

    def get_qvalues(self, states):
        # input is an array of states in numpy and outout is Qvals as numpy array
        states = torch.tensor(states, device=device, dtype=torch.float32)
        qvalues = self.forward(states)
        return qvalues.data.cpu().numpy()

    def sample_actions(self, qvalues):
    # sample actions from a batch of q_values using epsilon greedy policy
       epsilon = self.epsilon
       if qvalues.ndim == 1:
          qvalues = qvalues.reshape(1, -1)  # Reshape to make it a 2D array with batch size 1

       batch_size, n_actions = qvalues.shape
       random_actions = np.random.choice(n_actions, size=batch_size)
       best_actions = qvalues.argmax(axis=-1)
       should_explore = np.random.choice([0, 1], batch_size, p=[1 - epsilon, epsilon])
       return np.where(should_explore, random_actions, best_actions)




def evaluate(env, agent, n_games=1, greedy=False, t_max=10000):
	# used for evaluationing the trained agent for number of games = n_games and step in each game = t_max
	# returns the mean of sum of all rewards across n_games
	#TODO
    total_rewards = 0
    for i in range(n_games):
        state = env.reset()
        done = False
        t = 0

        while not done and t < t_max:
            q_values = agent.get_qvalues(torch.tensor(state, dtype=torch.float32).unsqueeze(0))

            if greedy:
                action = q_values.argmax().item()
            else:
                action = agent.sample_actions(q_values)[0]

            next_state, reward, done, i = env.step(action)
            total_rewards += reward
            state = next_state
            t += 1

    return total_rewards / n_games

In [ ]:
# Now we create a class ReplayBuffer. The object of this class is responsible for storing the buffer information based on the agent's action when we play the agent(i.e, current_State -> action -> next_state -> done_flag ->reward)
# For Deep Q Learning we sample information of size = 'batch_size' from the ReplayBuffer and return that information for training
# This buffer has a fixed size, set that to 10**6. remove previous information as new information is passed in the buffer


class ReplayBuffer:
	def __init__(self, size):
		#TODO
		# size is the maximum size that the buffer can hold
		self.size=size
		self.buffer=[]
		self.position=0


	def __len__(self):
		# no need to change
		return len(self.buffer)

	def add(self, state, action ,reward, next_state, done):
		experience=(state, action ,reward, next_state, done)
		if len(self.buffer)<self.size:
			self.buffer.append(experience)
		else:
			self.buffer[self.position] = experience

		self.position = (self.position + 1) % self.size
		#TODO
		# store the information passed in one call to add as 1 unit of informmation




	def sample(self, batch_size):
		#TODO
		# return a random sampling of 'batch_size' units of information
		batch=random.sample(self.buffer,min(batch_size,len(self.buffer)))
		return batch


In [ ]:
def play_and_record(start_state, agent, env, exp_replay, n_steps = 1):
	state = start_state
	for _ in range(n_steps):
		state_t = torch.tensor([state], dtype=torch.float32, device=device)
		qvalues_t = agent(state_t)
		qvalues = qvalues_t.cpu().detach().numpy()[0]
		action = agent.sample_actions(qvalues)[0]
		next_state, reward, done, _ = env.step(action)
		exp_replay.add(state, action, reward, next_state, done)
		if done:
			state = env.reset()
		else:
			state = next_state



	# use this function to make the agent play on the env and store the information in exp_replay which is an object of class ReplayBuffer
	# n_steps is the number of steps to be played in this function on one call
	#TODO
	# pass


def compute_td_loss(agent, target_network, device, batch_size, exp_replay ,gamma = 0.99,):
	states, actions, rewards, next_states, dones = zip(*exp_replay.sample(batch_size))
	states = torch.tensor(states,dtype=torch.float32).to(device)
	actions = torch.tensor(actions,dtype=torch.long).to(device)
	rewards = torch.tensor(rewards,dtype=torch.float32).to(device)
	next_states = torch.tensor(next_states,dtype=torch.float32).to(device)
	dones = torch.tensor(dones,dtype=torch.float32).to(device)

    # Compute the predicted Q-values of the actions using the agent
	q_values = agent(states)
	predicted_qvalues = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
	gamma=torch.tensor(gamma,dtype=torch.float32)

    # Compute the target Q-values of the actions using the target network
	with torch.no_grad():
		target_q_values = target_network(next_states)
		target_qvalues_of_actions = rewards + torch.mul(gamma,target_q_values.max(dim=1)[0]) * torch.logical_not(dones)

    # Compute the TD loss (Mean Squared Error)
	loss = torch.nn.MSELoss()(predicted_qvalues, target_qvalues_of_actions)
	return loss

	# Here agent is the one playing on the game and target_network is updates using agent after some fixed steps as is done in Deep Q Learning
	# sample 'batch_size' units of info stored in the exp_replay
	# Find the predicted_qvalues_of_actions using agent and target_qvalues_of_actions using target_network, find the loss based on these Mean Squared Error of these two
	# IMPORTANT NOTE : check the type of objects, U need to convert the actions, rewards, etc, to toch.tensors for backward propogation using pytorch
	#TODO
	# pass


In [ ]:
############# MAIN LOOP ###############
from tqdm import trange
from IPython.display import clear_output
import matplotlib.pyplot as plt
import torch.optim as optim

seed = 108
random.seed(108)
np.random.seed(108)
torch.manual_seed(108)


In [ ]:

##  setup environment using make_env function defined above
# find action_space and observation_space of the atari
# Use env_name = "BreakoutNoFrameskip-v4"
# Reset the environment before starting to train the agent and everytime the game ends (U will get a done flag which is a boolean representing whether the game has ended or not)
# TODO
env_name = "BreakoutNoFrameskip-v4"
env = make_env(env_name)
action_space = env.action_space
observation_space = env.observation_space
state_shape = observation_space.shape
n_actions = action_space.n
state = env.reset()
done = False

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


In [ ]:
# create agent from DQNAgent class the online network
# create target_network from DQNAgent class is updated after some fixed steps from agent
# Note initialise target network values from agent
# Create the online network (agent) and target network objects
agent = DQNAgent(observation_space.shape, action_space.n, epsilon=0.1)
target_network = DQNAgent(observation_space.shape, action_space.n, epsilon=0.1)

# Initialize the target network with the agent's values
target_network.load_state_dict(agent.state_dict())

# TODO


<All keys matched successfully>

In [ ]:
# created a ReplayBuffer object and saved some information in the object by playing the agent. It is better to populate some information in the Buffer, hence this step
#filling experience replay with some samples using full random policy
exp_replay = ReplayBuffer(10**6)
for i in range(4000):
    play_and_record(state, agent, env, exp_replay, n_steps=10**2)
    print( "Replay Buffer : i : ", i)
    if len(exp_replay) == 10**6:
        break
print(len(exp_replay))



In [ ]:
#setup some parameters for training
timesteps_per_epoch = 2
batch_size = 32

total_steps = 2 * 10**6

#Optimizer
optimizer = torch.optim.Adam(agent.parameters(), lr=2e-5)
# TODO - use Adam optimiser from torch with learning rate (lr) = 2*1e-5


In [ ]:
#setting exploration epsilon
start_epsilon = 0.1
end_epsilon = 0.05
eps_decay_final_step = 1 * 10**5

# setup spme frequency for logginf and updating target network
loss_freq = 20
refresh_target_network_freq = 100
eval_freq = 10000

# to clip the gradients
max_grad_norm = 5000

mean_rw_history = []
td_loss_history = []

SAVE_INTERVAL = 50000

from numpy import asarray
from numpy import savetxt


def epsilon_schedule(start_eps, end_eps, step, final_step):
    return start_eps + (end_eps-start_eps)*min(step, final_step)/final_step




In [ ]:
# TODO - reset the state of the environment before starting
state=env.reset()
## MAIN LOOP STARTING
for step in trange(total_steps + 1):

	#TODO update the exploration probability (epsilon) as time passes
		epsilon = epsilon_schedule(start_epsilon, end_epsilon, step, eps_decay_final_step)
		agent.epsilon = epsilon
	#TODO taking timesteps_per_epoch and update experience replay buffer, (use play_and_record)
		play_and_record(state, agent, env, exp_replay, n_steps=timesteps_per_epoch)
	#TODO compute loss
		loss = compute_td_loss(agent, target_network, device=device, batch_size=batch_size, exp_replay=exp_replay,gamma=0.99)
	#TODO Backward propogation and updating the network parameters
		optimizer.zero_grad()
		loss.backward()
		torch.nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)  # Clip gradients to avoid exploding gradients
		optimizer.step()
	# IMPORTANT NOTE : You only need to update the parameters of agent and not of target_network, that will be done according to refresh_target_network_freq. But Backward Propogation will take into account the target_network parameters as well. So use detach() method on target_network while calculating the loss. Google what it does and how to use !!


		if step % loss_freq == 0:
			td_loss_history.append(loss.data.cpu().item())


		if step % refresh_target_network_freq == 0:
        #TODO Load agent weights into target_network
			target_network.load_state_dict(agent.state_dict())

		if step % eval_freq == 0:
			mean_reward = evaluate(make_env(env_name, seed=step), agent, n_games=3, greedy=True, t_max=6000)
			mean_rw_history.append(mean_reward)

		print("mean_reward : ", mean_reward)

		clear_output(True)
		print("buffer size = %i, epsilon = %.5f" %
				(len(exp_replay), agent.epsilon))


		if step % SAVE_INTERVAL == 0 and step!= 0:
			print('Saving...')
			device = torch.device('cpu')
			torch.save(agent.state_dict(), f'model_{step}.pth')
			device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
		savetxt(f'reward_{step}.csv', np.array(mean_rw_history))



# savetxt('reward_final.csv', np.array(mean_rw_history))

final_score = evaluate(
  make_env(env_name),
  agent, n_games=1, greedy=True, t_max=10000
)
print('final score:', final_score)




  0%|          | 0/2000001 [00:00<?, ?it/s]<ipython-input-14-88a469e385d5>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states = torch.tensor(states, device=device, dtype=torch.float32)


mean_reward :  0.0


  0%|          | 1/2000001 [02:58<99087:46:44, 178.36s/it]

buffer size = 402, epsilon = 0.10000
mean_reward :  0.0


  0%|          | 2/2000001 [02:58<40951:26:43, 73.71s/it] 

...

buffer size = 17544, epsilon = 0.09571
mean_reward :  0.0


  0%|          | 8573/2000001 [1:30:06<413:07:35,  1.34it/s]

buffer size = 17546, epsilon = 0.09571
mean_reward :  0.0


  0%|          | 8574/2000001 [1:30:07<415:34:41,  1.33it/s]

buffer size = 17548, epsilon = 0.09571
mean_reward :  0.0
